<img src="https://github.com/facebookresearch/detectron2/raw/main/.github/Detectron2-Logo-Horz.svg" width="300" >

Detectron2 is Facebook AI Research's next generation library
that provides state-of-the-art detection and segmentation algorithms.
It is the successor of
[Detectron](https://github.com/facebookresearch/Detectron/)
and [maskrcnn-benchmark](https://github.com/facebookresearch/maskrcnn-benchmark/).
It supports a number of computer vision research projects and production applications in Facebook.

<div align="center">
  <img src="https://user-images.githubusercontent.com/1381301/66535560-d3422200-eace-11e9-9123-5535d469db19.png"/>
</div>

Sourced from [detectron2(github)](https://github.com/facebookresearch/Detectron/)

</br>

I wrote some ideas to improve model at [Please check my ideas to improve model🙏](https://www.kaggle.com/c/petfinder-pawpularity-score/discussion/295808).
The 3 idea is "Training by using crop dogs and cats by Object Detection images", and the 4 idea is "Separate dogs and cats then train".

Thus, in this code I did object detect as a pre-process of these two ideas, I used detectron2's COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x model to detect cats and dogs.
Then I did EDA, so I show my result.

The train.csv with object detect infomation like is [petfinder2-detect-info](https://www.kaggle.com/masaishi/petfinder2-detected-info)

</br>

### Result
Cats and dogs have different pawpularity distributions.
Cats mean is 35.62513274336283, and dog one is 41.24786324786325.
Thus, dogs' image can get beter pawpularity, so the 4 idea of making model for only cats and dogs may be good.


In [ ]:
import sys
import os

import albumentations
import pandas as pd
import numpy as np

import gc
from glob import glob
import pickle
import json
from pprint import pprint
import subprocess

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T

from tqdm import tqdm
import ast

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset


import cv2

from tqdm import tqdm

In [ ]:
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
# import detectron2
# from detectron2.utils.logger import setup_logger

# # import some common detectron2 utilities
# from detectron2 import model_zoo
# from detectron2.engine import DefaultPredictor
# from detectron2.config import CfgNode, LazyConfig, get_cfg, instantiate
# from detectron2.utils.visualizer import Visualizer
# from detectron2.data import MetadataCatalog, DatasetCatalog
# from detectron2.structures import Boxes, Instances, ROIMasks
# from detectron2.modeling import GeneralizedRCNN
# from detectron2.data import DatasetFromList
# from detectron2.checkpoint import DetectionCheckpointer


In [ ]:
DATA_DIR = '../input/petfinder-pawpularity-score'
IMG_PATH = '../input/petfinder-pawpularity-score'

In [ ]:
train_df=pd.read_csv(f'{DATA_DIR}/train.csv')
test_df=pd.read_csv(f'{DATA_DIR}/test.csv')

train_df['image_path'] = f'{IMG_PATH}/train/' + train_df['Id'] + '.jpg'
test_df['image_path'] = f'{IMG_PATH}/test/' + test_df['Id'] + '.jpg'

In [ ]:
# model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")

In [ ]:
# cfg = get_cfg()
# # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")

# predictor = DefaultPredictor(cfg)

In [ ]:
# !echo a | python -m pip download 'git+https://github.com/facebookresearch/detectron2.git'
# import pickle
# with open('mask_rcnn_X_101_32x8d_FPN_3x.pkl', 'wb') as p:
#     pickle.dump(predictor, p)

Showing how to make published petfinder2-detected-info dataset.
Below program is proceses of only 16 samples.

In [ ]:
# MetadataCatalog.get(cfg.DATASETS.TRAIN[0])

In [ ]:
# train_df['height'] = 0
# train_df['width'] = 0
# train_df['fields'] = ''
# train_df

In [ ]:
# cfg.INPUT.FORMAT

In [ ]:
# %%time

# plt.figure(figsize=(24, 24))

# for index in tqdm(range(len(train_df))):
#     # for index in tqdm(range(16)):
#     im = cv2.imread(train_df.loc[index, 'image_path'])
#     outputs = predictor(im)
    
# #     # Visualize
# #     v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
# #     out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# #     out = out.get_image()
# #     out = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)
# #     plt.subplot(4, 4, index+1)
# #     plt.imshow(out[:, :, ::-1])
    
#     # Add detect information to table
#     (height, width) =  outputs['instances'].image_size
#     fields = outputs['instances'].get_fields()
#     fields.pop('pred_masks')

#     train_df.loc[index, ['height', 'width', 'fields']] = height, width, fields
    
    

In [ ]:
# train_df.loc[0:15, ['image_path', 'height', 'width', 'fields']]

In [ ]:
# train_df.loc[0, 'fields']

In [ ]:
# train_df['Persons'] = 0
# train_df['Cats'] = 0
# train_df['Dogs'] = 0
# train_df

In [ ]:
# for index in tqdm(range(len(train_df))):
#     # for index in tqdm(range(16)):
#     train_df.loc[index, 'Persons'] = train_df.loc[index, 'fields']['pred_classes'].tolist().count(0)
#     train_df.loc[index, 'Cats'] = train_df.loc[index, 'fields']['pred_classes'].tolist().count(15)
#     train_df.loc[index, 'Dogs'] = train_df.loc[index, 'fields']['pred_classes'].tolist().count(16)
# train_df

In [ ]:
# train_df.to_csv("train.csv", index=False)

In [ ]:
# train_df['Cats'].sum()

In [ ]:
# train_df['Dogs'].sum()

In [ ]:
# train_df['Persons'].sum()

By doing that procese, I generate petfinder2-detected-info dataset.

# EDA of detect info

In [ ]:
DATA_DIR = '../input/petfinder2-detected-info'

train_df=pd.read_csv(f'{DATA_DIR}/train.csv')
train_df['image_path'] = f'{IMG_PATH}/train/' + train_df['Id'] + '.jpg'
train_df

In [ ]:
train_df['fields'] = train_df['fields'].apply(fields_to_dict)
train_df['fields']

In [ ]:
add_hwf(train_df)

In [ ]:
train_df_info['Cats'].sum()

In [ ]:
train_df_info['Dogs'].sum()

In [ ]:
train_df_info['Persons'].sum()

## Image Size

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(train_df['height'])

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(train_df['width'])

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(train_df['height'] + train_df['width'])

## Persons

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(train_df['Persons'])

In [ ]:
# Reason only plot 5 cats are the number of sample is too small over 5 cats.
print(train_df[train_df['Persons']==0]['Pawpularity'].mean(), train_df[train_df['Persons']>0]['Pawpularity'].mean())

plt.figure(figsize=(12, 6))
plt.hist(train_df[train_df['Persons']==0]['Pawpularity'], 50, alpha = 0.5, label=f'No person', density=True, stacked=True)
plt.hist(train_df[train_df['Persons']>0]['Pawpularity'], 50, alpha = 0.5, label=f'Persons', density=True, stacked=True)
plt.legend(loc='upper left')

In [ ]:
# Compare to labeled data
b_same_sum = sum((train_df['Human']) == (train_df['Persons'] > 0))
print(f'Both of labels is same {b_same_sum}')
print(f'Different labels {len(train_df) - b_same_sum}')

In [ ]:
# Human label is 0 but detectron2 detect human
specific_train = train_df[(train_df['Human'] == 0) & (train_df['Persons'] > 0)]

plt.figure(figsize=(24, 24))

# for index in tqdm(range(len(train_df))):
for index in range(16):
    im = cv2.imread(specific_train.iloc[index]['image_path'])
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.subplot(4, 4, index+1)
    plt.title(specific_train.iloc[index]['Pawpularity'])
    plt.imshow(im)

In [ ]:
# Human label is 1 but detectron2 can't detect human
specific_train = train_df[(train_df['Human'] == 1) & (train_df['Persons'] == 0)]

plt.figure(figsize=(24, 24))

# for index in tqdm(range(len(train_df))):
for index in range(16):
    im = cv2.imread(specific_train.iloc[index]['image_path'])
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.subplot(4, 4, index+1)
    plt.title(specific_train.iloc[index]['Pawpularity'])
    plt.imshow(im)

## Cats and Dogs

### Cats

In [ ]:
len(train_df[train_df['Cats'] > 0])

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(train_df['Cats'])

In [ ]:
# Reason only plot 5 cats are the number of sample is too small over 5 cats.
plt.figure(figsize=(12, 6))
for i in range(1, 4):
    print(len(train_df[train_df['Cats']==i]), train_df[train_df['Cats']==i]['Pawpularity'].mean())
    plt.hist(train_df[train_df['Cats']==i]['Pawpularity'], 50, alpha = 0.4, label=f'{i} cats', density=True, stacked=True)
plt.legend(loc='upper left')

In [ ]:
# Cat pictures
specific_train = train_df[train_df['Cats'] > 0]

plt.figure(figsize=(24, 24))

# for index in tqdm(range(len(train_df))):
for index in range(16):
    im = cv2.imread(specific_train.iloc[index]['image_path'])
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.subplot(4, 4, index+1)
    plt.title(specific_train.iloc[index]['Pawpularity'])
    plt.imshow(im)

### Dogs

In [ ]:
len(train_df[train_df['Dogs'] > 0])

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(train_df['Dogs'])

In [ ]:
plt.figure(figsize=(12, 6))
for i in range(1, 4):
    print(len(train_df[train_df['Dogs']==i]), train_df[train_df['Dogs']==i]['Pawpularity'].mean())
    plt.hist(train_df[train_df['Dogs']==i]['Pawpularity'], 50, alpha = 0.4, label=f'{i} dogs', density=True, stacked=True)
plt.legend(loc='upper left')

In [ ]:
# Cat pictures
specific_train = train_df[train_df['Dogs'] > 0]

plt.figure(figsize=(24, 24))

# for index in tqdm(range(len(train_df))):
for index in range(16):
    im = cv2.imread(specific_train.iloc[index]['image_path'])
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.subplot(4, 4, index+1)
    plt.title(specific_train.iloc[index]['Pawpularity'])
    plt.imshow(im)

## Compare Dog and Cat

In [ ]:
print('Cats and Dog mean')
print(train_df[train_df['Cats']>0]['Pawpularity'].mean(), train_df[train_df['Dogs']>0]['Pawpularity'].mean())

In [ ]:
# Normalized histgram
plt.figure(figsize=(12, 6))
plt.hist(train_df[train_df['Cats']>0]['Pawpularity'], 50, alpha = 0.5, label='Cats', density=True, stacked=True)
plt.hist(train_df[train_df['Dogs']>0]['Pawpularity'], 50, alpha = 0.5, label='Dogs', density=True, stacked=True)
plt.legend(loc='upper left')

In [ ]:
# The number of animals
train_df['Animals'] = train_df['Cats'] + train_df['Dogs']

In [ ]:
plt.figure(figsize=(12, 6))
for i in range(0, 5):
    print(len(train_df[train_df['Animals']==i]), train_df[train_df['Animals']==i]['Pawpularity'].mean())
    plt.hist(train_df[train_df['Animals']==i]['Pawpularity'], 50, alpha = 0.4, label=f'{i} animals', density=True, stacked=True)
plt.legend(loc='upper left')

In [ ]:
# Showing pictures
for i in range(0, 8):
    # Cat pictures
    specific_train = train_df[train_df['Animals']==i]
    
    plt.figure(figsize=(24, 12))
    plt.suptitle(f'{i} animals')

    # for index in tqdm(range(len(train_df))):
    for index in range(8):
        im = cv2.imread(specific_train.iloc[index]['image_path'])
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        
        plt.subplot(1, 8, index+1)
        plt.title(specific_train.iloc[index]['Pawpularity'])
        plt.imshow(im)

In [ ]:
train_df['fields'][0]

In [ ]:
plt.figure(figsize=(24, 24))
for index in range(16):
    im = cv2.imread(train_df.loc[index, 'image_path'])
    outputs = train_df.loc[index, 'fields']
    outputs = predictor(im)
    print(outputs)
    print(type(outputs))
    break
    
    # Visualize
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    out = out.get_image()
    out = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)
    plt.subplot(4, 4, index+1)
    plt.title(train_df.iloc[index]['Pawpularity'])
    plt.imshow(out[:, :, ::-1])

In [ ]:
import json

In [ ]:

from detectron2.structures import Boxes, Instances, ROIMasks

train_df['fields'].apply(Instances)

In [ ]:
train_df['fields'][0]

In [ ]:
train_df['fields'][0][1:-1]

In [ ]:
train_df['fields']['']

In [ ]:
test = ast.literal_eval(train_df['fields'][0])

In [ ]:

ast.literal_eval(json.dumps(train_df['fields'][0], ensure_ascii=False).encode('utf8'))


In [ ]:
test = detectron2.structures.instances.Instances(image_size=[train_df['height'][0], train_df['width'][0]], train_df['fields'][0])

In [ ]:
test

In [ ]:
test.get('pred_boxes')